In [1]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
import time
import pprint
from multiprocessing import Pool
import numpy as np
import pandas as pd
from math import sqrt
import tensorflow as tf

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
np.random.seed(42)
train_set = open("ml-1m/ratings.dat").readlines()
train_set = np.random.permutation(train_set)[:100]
train_threshold = int(0.75 * len(train_set))
# train_indices = []
# test_indices = []
train_user_indices = []
train_item_indices = []
test_user_indices = []
test_item_indices = []
train_ratings = []
test_ratings = []

data = defaultdict(dict)
user2id = {}
item2id = {}
index_user = 0
index_item = 0

for i, line in enumerate(train_set):
    user = line.split("::")[0]
    item = line.split("::")[1]
    rating = line.split("::")[2]
    
    try:
        user_id = user2id[user]
    except KeyError:
        user_id = index_user
        user2id[user] = index_user
        index_user += 1
    try:
        item_id = item2id[item]
    except KeyError:
        item_id = index_item
        item2id[item] = index_item
        index_item += 1
        
    if i < train_threshold:
    #    train_indices.append((user_id, item_id))
        train_user_indices.append(user_id)
        train_item_indices.append(item_id)
        train_ratings.append(int(rating))
        data[user_id].update(dict(zip([item_id], [int(rating)])))
    else:
    #    test_indices.append((user_id, item_id))
        test_user_indices.append(user_id)
        test_item_indices.append(item_id)
        test_ratings.append(int(rating))

In [4]:
len(train_ratings), len(test_ratings)

(75, 25)

In [14]:
def ratings(dataset):
    for user, r in data.items():
        for item, rating in r.items():
            yield user, item, rating

In [27]:
n_users = len(user2id)
n_items = len(item2id)
n_factors = 100
user_list = np.array(list(data.keys()), dtype=np.int32)

In [32]:
aa = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "rating", "timestamp"])
aa = aa.filter(regex="user|item|rating")

In [36]:
aa.head()

,user,item,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [39]:
aa[aa.user == 1].to_dict()

{'item': {0: 1193,
  1: 661,
  2: 914,
  3: 3408,
  4: 2355,
  5: 1197,
  6: 1287,
  7: 2804,
  8: 594,
  9: 919,
  10: 595,
  11: 938,
  12: 2398,
  13: 2918,
  14: 1035,
  15: 2791,
  16: 2687,
  17: 2018,
  18: 3105,
  19: 2797,
  20: 2321,
  21: 720,
  22: 1270,
  23: 527,
  24: 2340,
  25: 48,
  26: 1097,
  27: 1721,
  28: 1545,
  29: 745,
  30: 2294,
  31: 3186,
  32: 1566,
  33: 588,
  34: 1907,
  35: 783,
  36: 1836,
  37: 1022,
  38: 2762,
  39: 150,
  40: 1,
  41: 1961,
  42: 1962,
  43: 2692,
  44: 260,
  45: 1028,
  46: 1029,
  47: 1207,
  48: 2028,
  49: 531,
  50: 3114,
  51: 608,
  52: 1246},
 'rating': {0: 5,
  1: 3,
  2: 3,
  3: 4,
  4: 5,
  5: 3,
  6: 5,
  7: 5,
  8: 4,
  9: 4,
  10: 5,
  11: 4,
  12: 4,
  13: 4,
  14: 5,
  15: 4,
  16: 3,
  17: 4,
  18: 5,
  19: 4,
  20: 3,
  21: 3,
  22: 5,
  23: 5,
  24: 3,
  25: 5,
  26: 4,
  27: 4,
  28: 4,
  29: 3,
  30: 4,
  31: 4,
  32: 4,
  33: 4,
  34: 4,
  35: 4,
  36: 5,
  37: 5,
  38: 4,
  39: 5,
  40: 5,
  41: 5,
  42: 4

In [41]:
data[1].keys(), data[1].values()

(dict_keys([1]), dict_values([5]))

In [20]:
aa.iloc[0].to_dict().values()

dict_values([1, 1193, 5])

In [21]:
def ratings2(dataset):
    for user, item, rating in zip(dataset.user, dataset.item, dataset.rating):
        yield user, item, rating

In [68]:
aa = np.arange(16).reshape(4,4)
aa

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [96]:
np.take_along_axis(aa[[0,2]], np.array([[0,2]]), axis=1)

array([[ 0,  2],
       [ 8, 10]])

In [100]:
np.take(aa[[0,2]], [0,2], axis=1)

array([[ 0,  2],
       [ 8, 10]])

In [104]:
aa[[0,2]][:, [0,2]]

array([[ 0,  2],
       [ 8, 10]])

In [113]:
aa[[0,2,2], [1,2,3]]

array([ 1, 10, 11])

In [120]:
np.array([i for i in range(10)])

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [118]:
np.take(aa, list(zip(index1, index2)))

array([[0, 1],
       [2, 2],
       [2, 3]])

In [115]:
np.where(aa == 10)

(array([2]), array([2]))

## Collaborative Filtering

In [ ]:
# UNDO :  common calculate   raw_id2inner_id  pearson_similarity  predict_ratings

In [5]:
def common(x1,x2):
    # find common ratings
    common = (x1 != 0) & (x2 != 0)
    new_x1 = x1[common]
    new_x2 = x2[common]
    return new_x1, new_x2

In [6]:
x = np.array([1,2,3,0])
x1 = np.array([1,0, 4, 6])
c = (x != 0) & (x1 != 0)
x[c], x1[c]

(array([1, 3]), array([1, 4]))

In [2]:
def common(x1, x2):
    common = (x1 != 0) & (x2 != 0)
    new_x1 = x1[common]
    new_x2 = x2[common]
    return new_x1, new_x2

In [16]:
x1 = np.array([4,0,5,5])
x2 = np.array([2,1,3,5])
common(x1, x2)

(array([4, 5, 5]), array([2, 3, 5]))

In [17]:
list(zip(common(x1, x2)[0], common(x1, x2)[1]))

[(4, 2), (5, 3), (5, 5)]

In [18]:
numerator = 0
denominator1 = 0
denominator2 = 0
mean1 = np.mean(common(x1, x2)[0])
mean2 = np.mean(common(x1, x2)[1])
for i, j in zip(common(x1, x2)[0], common(x1, x2)[1]):
    numerator += (i - mean1) * (j - mean2)
    denominator1 += (i - mean1) ** 2
    denominator2 += (j - mean2) ** 2

numerator / (np.sqrt(denominator1) * np.sqrt(denominator2))

0.7559289460184546

## matrix factorization

In [ ]:
example = defaultdict(dict)
for key in range(200):
#    key = str(key)
    example[key] = data[key]
# data = example

In [9]:
a = tf.constant(np.array([[1,2,3,0,2], [1,0,3,0,3]]), dtype=tf.float32)
zero = tf.constant(0, dtype=tf.float32)  ## -1
mask = tf.not_equal(a, zero)
# indices = tf.reshape(tf.where(mask), shape=[-1])
indices = tf.where(mask)
c = tf.gather_nd(a, indices)
d = tf.boolean_mask(a, mask)

sess = tf.InteractiveSession()
sess.run([mask, indices, c, d])

[array([[ True,  True,  True, False,  True],
        [ True, False,  True, False,  True]]), array([[0, 0],
        [0, 1],
        [0, 2],
        [0, 4],
        [1, 0],
        [1, 2],
        [1, 4]]), array([1., 2., 3., 2., 1., 3., 3.], dtype=float32), array([1., 2., 3., 2., 1., 3., 3.], dtype=float32)]

In [16]:
def func(a):
    return a[0] + a[1] + a[2]

aa = np.array(train_user_indices)[:10]
bb = np.array(train_item_indices)[:10]
cc = np.array(train_ratings)[:10]
nu = tf.map_fn(func, [aa, bb, cc], dtype=tf.int64)
sess = tf.InteractiveSession()
sess.run(nu)

array([ 2,  7,  8, 10,  9, 14, 17, 17, 17, 19])

In [17]:
aa = np.array([[1,2,3,0,0], [4,5,6,7,8]], dtype=np.int32)
oo = tf.nn.embedding_lookup(aa, 0)
zero = tf.constant(0, dtype=tf.int32)
mask = tf.not_equal(oo, zero)
u_items_mask = tf.boolean_mask(oo, mask)
# ii = tf.gather(oo, u_items_mask)
sess = tf.InteractiveSession()
sess.run([u_items_mask, tf.size(u_items_mask)])

[array([1, 2, 3], dtype=int32), 3]

In [18]:
train_data = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_items = list(data[u].keys())
    train_data[u] = np.array(u_items + [0] * (n_items - len(u_items)))

pseudo_data = tf.placeholder(tf.int32, shape=[n_users, n_items])
yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))

def nu_func(user):
    u_items = tf.nn.embedding_lookup(pseudo_data, user)
    zero = tf.constant(0, dtype=tf.int32)
    mask = tf.not_equal(u_items, zero)
    u_items_mask = tf.boolean_mask(u_items, mask)
    return tf.reduce_sum(tf.gather(yj, u_items_mask), axis=0)

user_list = np.array(list(data.keys()), dtype=np.int32)
nu = tf.map_fn(nu_func, user_list, dtype=tf.float32)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run([nu, tf.shape(nu)], feed_dict={pseudo_data: train_data})

[array([[-0.0599637 ,  0.10894931,  0.02212552, ...,  0.06748607,
         -0.17401433, -0.09068567],
        [ 0.01508748,  0.01031978,  0.00728436, ...,  0.02157799,
         -0.03474513, -0.06860533],
        [ 0.0499043 ,  0.04905506, -0.09098595, ...,  0.2249795 ,
          0.03475139, -0.01830734],
        ...,
        [-0.02819956, -0.00509941,  0.00060968, ..., -0.02477363,
         -0.00529699,  0.01257202],
        [-0.01476342, -0.00543445, -0.00674025, ..., -0.0633116 ,
         -0.00543289,  0.00431837],
        [-0.0382369 ,  0.04862677, -0.00541936, ...,  0.00024505,
          0.03966409,  0.00907592]], dtype=float32),
 array([6040,  100], dtype=int32)]

### ALS

In [13]:
aa = np.array([1,2,3]).reshape(1,-1)
bb = np.array([4,5,6]).reshape(1, -1)
aa.T.dot(aa) + bb.T.dot(bb)

array([[17, 22, 27],
       [22, 29, 36],
       [27, 36, 45]])

In [14]:
cc = np.array([[1,2,3],[4,5,6]])
cc.T.dot(cc)

array([[17, 22, 27],
       [22, 29, 36],
       [27, 36, 45]])

In [16]:
aa = np.array([1,2,3])
aa = np.expand_dims(aa, axis=1)
bb = np.array([[1,1,1,1], [2,2,2,2], [3,3,3,3]])
aa * bb, np.sum(np.multiply(aa, bb), axis=0)

(array([[1, 1, 1, 1],
        [4, 4, 4, 4],
        [9, 9, 9, 9]]), array([14, 14, 14, 14]))

## Factorization Meets the Neighborhood

In [19]:
aa = pd.read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "rating", "timestamp"])
aa = aa.filter(regex="user|item|rating")

In [20]:
aa.iloc[0].to_dict().values()

dict_values([1, 1193, 5])

In [21]:
def ratings2(dataset):
    for user, item, rating in zip(dataset.user, dataset.item, dataset.rating):
        yield user, item, rating

In [25]:
train_data = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_items = list(data[u].keys())
    train_data[u] = np.array(u_items + [-1] * (n_items - len(u_items)))

In [31]:
aa = tf.placeholder(tf.int32, shape=[None, len(item2id)])
oo = tf.nn.embedding_lookup(aa, 1)
zero = tf.constant(-1, dtype=tf.int32)
mask = tf.not_equal(oo, zero)
u_items_mask = tf.boolean_mask(oo, mask)
sess = tf.InteractiveSession()
sess.run(u_items_mask, feed_dict={aa: train_data})

array([   1, 1371, 1992,  202,  139,  269,   15, 1176,  533, 2200, 1124,
        282,  591,  989,  224,  737, 1763,  804,  549, 1063, 3049, 1519,
       1457, 2696, 2345, 1019], dtype=int32)

In [32]:
aa = tf.constant([[1,2,3], [4,5,6]])
bb = tf.gather(tf.gather(aa, 0), [1,1])
test_user = tf.placeholder(tf.int32, shape=[None])
test_item = tf.placeholder(tf.int32, shape=[None])
# elems = (np.array([1,2,3]), np.array([4,5,6]))
def func(x):
    q = x[0]
    r = x[1]
    return q * r

cc = tf.map_fn(func, (test_user, test_item), dtype=tf.int32)

# lambda x: x[0] + x[1]
# elems = (np.array([1, 2, 3], dtype=np.int32), np.array([-1, 1, -1], dtype=np.int32))
# alternate = tf.map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int32)
sess = tf.InteractiveSession()
sess.run(cc, feed_dict={test_user: [1,2,3], test_item: [9,5,6]})

array([ 9, 10, 18], dtype=int32)

In [7]:
import itertools
list(itertools.product([1,2,3], [4,5,6]))

[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]

In [10]:
%%time
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c_u = np.zeros((n_users, n_items))
for u, i in zip(train_user_indices, train_item_indices):
    u_items = list(data[u].keys())
    c_ij = tf.reduce_sum(tf.gather(tf.gather(c, i), u_items)) / \
           tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
    c_u[u, i] = c_ij
    
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(c_u)

KeyboardInterrupt: 

#### embedding_lookup

In [96]:
aa = tf.constant([[1,2,3,4,5], [4,5,6,3,9]])
bb = tf.nn.embedding_lookup(aa, [[0,1], [1,1]])
user = tf.gather(bb, 0)
yj = tf.reduce_sum(user ,axis=0)
sess = tf.InteractiveSession()
sess.run([user, yj])

[array([[1, 2, 3, 4, 5],
        [4, 5, 6, 3, 9]], dtype=int32),
 array([ 5,  7,  9,  7, 14], dtype=int32)]

In [11]:
aa = tf.constant([[1,2,3], [4,5,6]])
bb = tf.nn.embedding_lookup(aa, [[0,1]])
sess = tf.InteractiveSession()
sess.run(bb)

array([[[1, 2, 3],
        [4, 5, 6]]], dtype=int32)

#### embedding_lookup_sparse

In [6]:
aa = np.arange(10).reshape(2,5).astype(float)
aa

array([[0., 1., 2., 3., 4.],
       [5., 6., 7., 8., 9.]])

In [7]:
indices = [[0,0], [0,0], [1,1], [2,1], [2,2], [2,2]]
values = [0,1,1,1,1,1]
dense_shape = [2,15]
st = tf.SparseTensor(indices, values, dense_shape)
# aa = tf.constant([[1,2,3], [4,5,6]], dtype=tf.float32)
bb = tf.nn.embedding_lookup_sparse(aa, st, sp_weights=None)
sess = tf.InteractiveSession()
sess.run(bb)

array([[2.5, 3.5, 4.5, 5.5, 6.5],
       [5. , 6. , 7. , 8. , 9. ],
       [5. , 6. , 7. , 8. , 9. ]])

In [84]:
aa = np.arange(27).reshape(3,3,3).astype(float)
aa

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.]],

       [[ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.]],

       [[18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.]]])

In [86]:
indices = [[0,0], [0,1], [0,1], [2,1], [2,2], [3,2]]
values = [1,2,1,1,1,2]
dense_shape = [2,15]
st = tf.SparseTensor(indices, values, dense_shape)
# aa = tf.constant([[1,2,3], [4,5,6]], dtype=tf.float32)
bb = tf.nn.embedding_lookup_sparse(aa, st, sp_weights=None)
sess = tf.InteractiveSession()
sess.run(bb)

array([[[12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.]],

       [[18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.]]])

In [108]:
N = [[] for u in range(n_users)]
for u, i, in zip(train_user_indices, train_item_indices):
    N[u].append(i)

sparse = {'indices': [], 'values': []}
for i, user in enumerate(N):
    for j, item in enumerate(user):
        sparse['indices'].append((i, j))
        sparse['values'].append(item)
        
sparse['dense_shape'] = (n_users, n_items)
implicit_feedback = tf.SparseTensor(**sparse)

yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
yjs = tf.nn.embedding_lookup_sparse(yj, implicit_feedback, sp_weights=None, combiner='sqrtn')
yju = tf.gather(yjs, [2,2,3])
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run([yju, tf.shape(yju)])

[array([[-0.00455127,  0.01736358, -0.0011284 ,  0.00255089, -0.00957593,
         -0.0020925 , -0.01266749, -0.00326786,  0.00396166, -0.00969134,
         -0.00675968, -0.00487921, -0.00627048,  0.00205449, -0.00103079,
          0.00094458, -0.00496926, -0.0083558 ,  0.00159714, -0.01686201,
         -0.00210911, -0.00651083, -0.00908738,  0.01984688,  0.00605965,
         -0.01005583, -0.01515545, -0.00346797, -0.00997632, -0.01489756,
          0.00344885, -0.00133769,  0.00312323,  0.01420122,  0.01937641,
         -0.00676895,  0.00130137,  0.00923016,  0.00342921,  0.00403274,
         -0.01719966,  0.00515556,  0.00074134, -0.00503118,  0.00809505,
          0.00481872, -0.00567423,  0.00239443,  0.00779041,  0.02053146,
          0.00215945,  0.00928877, -0.00847241, -0.00300206, -0.00708658,
          0.00164711,  0.00276376,  0.00022526, -0.01156578, -0.00452333,
         -0.00070281, -0.00081126, -0.00629355,  0.01388548,  0.02596579,
          0.00765122,  0.02379264,  0.

#### embedding_lookup_sparse - FM

In [213]:
w = np.arange(10).reshape(-1,1).astype(float)
w

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]])

In [214]:
indices = [[0], [0], [1], [2], [2], [2]]
values = [0,1,2,3,7,8]
dense_shape = [1]
st = tf.SparseTensor(indices, values, dense_shape)
sw = tf.SparseTensor(indices, [1,2,1,1,1,2], dense_shape)
bb = tf.nn.embedding_lookup_sparse(w, st, sp_weights=sw, combiner="sum")
sess = tf.InteractiveSession()
sess.run(bb)

array([[ 2.],
       [ 2.],
       [26.]])

In [46]:
sparse = {'indices': [], 'values': []}
for i, sample in enumerate(indices):
    for j, ind in enumerate(sample):
        sparse['indices'].append((i, j))
        sparse['values'].append(ind)
        
sparse['dense_shape'] = (100, 10)
sp_id = tf.SparseTensor(**sparse)

In [53]:
sp_weight = tf.SparseTensor(sparse['indices'], value.flatten(), sparse['dense_shape'])

In [54]:
res = tf.nn.embedding_lookup_sparse(v, sp_id, sp_weights=sp_weight, combiner="sum")
sess = tf.InteractiveSession()
sess.run(res)

array([[163],
       [165],
       [144]])

In [68]:
v = np.arange(1000).reshape(100, 10)
v[:10]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [140]:
indices = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
indices

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [141]:
value = np.array([[1, 1, 2], [1, 1, 2], [1, 1, 2]])
value

array([[1, 1, 2],
       [1, 1, 2],
       [1, 1, 2]])

In [66]:
sparse = {'indices': [], 'values': []}
for i, sample in enumerate(indices):
    for j, ind in enumerate(sample):
        sparse['indices'].append((i, j))
        sparse['values'].append(ind)
        
sparse['dense_shape'] = (100, 10)
sp_id = tf.SparseTensor(**sparse)

In [73]:
sparse_sw = {'indices': sparse['indices'], 'values': value.flatten(), 'dense_shape': sparse['dense_shape']}
sw = tf.SparseTensor(**sparse_sw)

In [71]:
res = tf.nn.embedding_lookup_sparse(v, sp_id, sp_weights=None, combiner="sum")
sess = tf.InteractiveSession()
sess.run(res)

array([[ 30,  33,  36,  39,  42,  45,  48,  51,  54,  57],
       [120, 123, 126, 129, 132, 135, 138, 141, 144, 147],
       [210, 213, 216, 219, 222, 225, 228, 231, 234, 237]])

In [74]:
res = tf.nn.embedding_lookup_sparse(v, sp_id, sp_weights=sw, combiner="sum")
sess = tf.InteractiveSession()
sess.run(res)

array([[ 50,  54,  58,  62,  66,  70,  74,  78,  82,  86],
       [170, 174, 178, 182, 186, 190, 194, 198, 202, 206],
       [290, 294, 298, 302, 306, 310, 314, 318, 322, 326]])

In [230]:
v = np.arange(1000).reshape(100, 10)
v[:5]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]])

In [231]:
indices = np.array([[0, 2, 19, 20, 30, 90], [0, 2, 19, 22, 40, 80], [0, 2, 18, 22, 40, 60]])
indices

array([[ 0,  2, 19, 20, 30, 90],
       [ 0,  2, 19, 22, 40, 80],
       [ 0,  2, 18, 22, 40, 60]])

In [220]:
value = np.array([[1, 2, 1, 1, 1, 1], [1, 2, 1, 1, 1, 1], [1, 2, 1, 1, 1, 1]])
value

array([[1, 2, 1, 1, 1, 1],
       [1, 2, 1, 1, 1, 1],
       [1, 2, 1, 1, 1, 1]])

In [232]:
feature_indices = tf.placeholder(tf.int64, shape=[None, 6])
sparse_indices = tf.placeholder(tf.int64, shape=[None, 1])

In [233]:
sparse = {'indices': sparse_indices, 'values': tf.reshape(feature_indices, [-1]), 'dense_shape': [1]}
tt = tf.SparseTensor(**sparse)

In [239]:
sparse_sw = {'indices': sparse['indices'], 'values': tf.reshape(value, [-1]), 'dense_shape': sparse['dense_shape']}
sw = tf.SparseTensor(**sparse_sw)

In [240]:
def build_sparse_indices(indices):
    si = []
    for i in range(indices.shape[0]):
        si.append(np.zeros([indices.shape[1], 1], dtype=np.int64) + i)
    return np.concatenate(si)

In [241]:
res = tf.nn.embedding_lookup_sparse(v, tt, sp_weights=None, combiner="sum")
sess = tf.InteractiveSession()
sess.run(res, {feature_indices: indices, 
               sparse_indices: build_sparse_indices(indices)})

array([[1610, 1616, 1622, 1628, 1634, 1640, 1646, 1652, 1658, 1664],
       [1630, 1636, 1642, 1648, 1654, 1660, 1666, 1672, 1678, 1684],
       [1420, 1426, 1432, 1438, 1444, 1450, 1456, 1462, 1468, 1474]])

In [242]:
res = tf.nn.embedding_lookup_sparse(v, tt, sp_weights=sw, combiner="sum")
sess = tf.InteractiveSession()
sess.run(res, {feature_indices: indices, 
               sparse_indices: build_sparse_indices(indices)})

array([[1630, 1637, 1644, 1651, 1658, 1665, 1672, 1679, 1686, 1693],
       [1650, 1657, 1664, 1671, 1678, 1685, 1692, 1699, 1706, 1713],
       [1440, 1447, 1454, 1461, 1468, 1475, 1482, 1489, 1496, 1503]])

In [227]:
feature_indices.get_shape()

TensorShape([Dimension(None), Dimension(6)])

## Dataset API

In [34]:
dataset = tf.data.Dataset.from_tensor_slices((train_user_indices, 
                                              train_item_indices, 
                                              train_ratings))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while 1:
            print(sess.run(one_element)[0], '-', sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("End!")

0 - (1, 1, 5)
2 - (3, 3, 4)
4 - (5, 5, 4)
6 - (7, 7, 3)
8 - (9, 9, 1)
10 - (11, 11, 4)
12 - (13, 13, 5)
14 - (15, 12, 3)
16 - (17, 16, 5)
18 - (19, 18, 2)
20 - (21, 20, 2)
22 - (23, 22, 4)
24 - (25, 23, 4)
26 - (27, 25, 5)
28 - (29, 27, 5)
30 - (29, 29, 4)
31 - (32, 31, 5)
33 - (34, 33, 5)
35 - (36, 35, 4)
37 - (38, 37, 2)
39 - (40, 39, 4)
41 - (42, 41, 3)
43 - (44, 43, 4)
45 - (46, 45, 2)
47 - (48, 47, 5)
49 - (50, 49, 3)
51 - (52, 51, 4)
53 - (54, 53, 4)
55 - (56, 55, 4)
57 - (58, 57, 2)
59 - (60, 59, 3)
61 - (62, 61, 3)
63 - (64, 10, 5)
65 - (66, 64, 4)
67 - (68, 66, 5)
69 - (70, 68, 4)
71 - (72, 23, 3)
End!


In [20]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "user": train_user_indices, 
        "item": train_item_indices, 
        "rating": train_ratings
    })
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while 1:
            print(sess.run(one_element)["user"], '-', sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("End!")

0 - {'rating': 5, 'item': 1, 'user': 1}
2 - {'rating': 4, 'item': 3, 'user': 3}
4 - {'rating': 4, 'item': 5, 'user': 5}
6 - {'rating': 3, 'item': 7, 'user': 7}
8 - {'rating': 1, 'item': 9, 'user': 9}
10 - {'rating': 4, 'item': 11, 'user': 11}
12 - {'rating': 5, 'item': 13, 'user': 13}
14 - {'rating': 3, 'item': 12, 'user': 15}
16 - {'rating': 5, 'item': 16, 'user': 17}
18 - {'rating': 2, 'item': 18, 'user': 19}
20 - {'rating': 2, 'item': 20, 'user': 21}
22 - {'rating': 4, 'item': 22, 'user': 23}
24 - {'rating': 4, 'item': 23, 'user': 25}
26 - {'rating': 5, 'item': 25, 'user': 27}
28 - {'rating': 5, 'item': 27, 'user': 29}
30 - {'rating': 4, 'item': 29, 'user': 29}
31 - {'rating': 5, 'item': 31, 'user': 32}
33 - {'rating': 5, 'item': 33, 'user': 34}
35 - {'rating': 4, 'item': 35, 'user': 36}
37 - {'rating': 2, 'item': 37, 'user': 38}
39 - {'rating': 4, 'item': 39, 'user': 40}
41 - {'rating': 3, 'item': 41, 'user': 42}
43 - {'rating': 4, 'item': 43, 'user': 44}
45 - {'rating': 2, 'item':

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "user": train_user_indices, 
        "item": train_item_indices, 
        "rating": train_ratings
    })
dataset = dataset.shuffle(100).batch(10)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while 1:
            print(sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("End!")

{'user': array([51, 63, 58, 42, 65, 27, 46, 29, 71, 60], dtype=int32), 'rating': array([3, 1, 2, 3, 3, 5, 2, 5, 4, 3], dtype=int32), 'item': array([50, 62, 57, 41, 63, 25, 45, 27, 61, 59], dtype=int32)}
{'user': array([48, 55, 32, 29, 30, 26, 68, 69, 37,  8], dtype=int32), 'rating': array([5, 3, 5, 4, 4, 5, 5, 2, 5, 1], dtype=int32), 'item': array([47, 54, 31, 29, 28, 24, 66, 67, 36,  8], dtype=int32)}
{'user': array([ 5, 31, 57, 24, 21, 25, 33, 36, 44, 49], dtype=int32), 'rating': array([4, 2, 5, 4, 2, 4, 5, 4, 4, 4], dtype=int32), 'item': array([ 5, 30, 56, 13, 20, 23, 32, 35, 43, 48], dtype=int32)}
{'user': array([40, 28, 47, 13, 61, 11, 41, 45, 18, 12], dtype=int32), 'rating': array([4, 5, 5, 5, 5, 4, 5, 1, 2, 4], dtype=int32), 'item': array([39, 26, 46, 13, 60, 11, 40, 44, 17, 12], dtype=int32)}
{'user': array([ 2, 20, 52, 35, 16, 73, 38, 15,  3,  7], dtype=int32), 'rating': array([4, 5, 4, 3, 4, 4, 2, 3, 4, 3], dtype=int32), 'item': array([ 2, 19, 51, 34, 15, 69, 37, 12,  3,  7],

In [69]:
dataset = tf.data.Dataset.from_tensor_slices({'user': train_user_indices, 
                                              'item': train_item_indices, 
                                              'rating': train_ratings})
dataset = dataset.shuffle(len(train_ratings)).batch(16).repeat(2)
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
batch = iterator.get_next()
sess = tf.InteractiveSession()
sess.run(iterator.make_initializer(dataset))
try:
    while True:
        pprint.pprint(sess.run(batch))
except tf.errors.OutOfRangeError:
    pass

{'item': array([64, 13, 69, 49,  1, 22, 10, 12, 23, 16, 59, 23, 57, 41, 44, 37]),
 'rating': array([4, 4, 4, 3, 5, 4, 3, 4, 3, 5, 3, 4, 2, 3, 1, 2]),
 'user': array([66, 24, 73, 50,  1, 23, 10, 12, 72, 17, 60, 25, 58, 42, 45, 38])}
{'item': array([19, 38, 39, 43, 15, 26,  3, 35, 42, 17, 52, 60, 34, 11, 54, 47]),
 'rating': array([5, 4, 4, 4, 4, 5, 4, 4, 3, 2, 5, 5, 3, 4, 3, 5]),
 'user': array([20, 39, 40, 44, 16, 28,  3, 36, 43, 18, 53, 61, 35, 11, 55, 48])}
{'item': array([18, 58, 46, 63, 67, 25, 20, 30, 50, 61,  7, 32, 14, 56, 68, 55]),
 'rating': array([2, 3, 5, 3, 2, 5, 2, 2, 3, 3, 3, 5, 3, 5, 4, 4]),
 'user': array([19, 59, 47, 65, 69, 27, 21, 31, 51, 62,  7, 33, 14, 57, 70, 56])}
{'item': array([24, 66, 61, 53, 12, 29,  8, 13, 45, 27,  4,  9, 36, 31,  6,  5]),
 'rating': array([5, 5, 4, 4, 3, 4, 1, 5, 2, 5, 1, 1, 5, 5, 5, 4]),
 'user': array([26, 68, 71, 54, 15, 29,  8, 13, 46, 29,  4,  9, 37, 32,  6,  5])}
{'item': array([28, 65, 10, 33, 40,  0,  2, 21, 48, 62, 51]),
 'rating':

In [2]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while 1:
            print(sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("End!")

1.0
2.0
3.0
4.0
5.0
End!


In [13]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": np.array([1.0, 2.0, 3.0, 4.0, 5.0]), 
        "b": np.random.uniform(size=(5,2))
    })

In [14]:
dataset.output_types, dataset.output_shapes

({'a': tf.float64, 'b': tf.float64},
 {'a': TensorShape([]), 'b': TensorShape([Dimension(2)])})

In [19]:
c = tf.constant([4.0, 2.0, 3.0])
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
sess.run([one_element['b'], tf.gather(c, tf.cast(one_element['a'], tf.int32))])

[array([0.83989501, 0.6755826 ]), 2.0]

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 2)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
try:
    while True:
        print(sess.run(one_element))
except tf.errors.OutOfRangeError:
    print("End!")

3.0
4.0
5.0
6.0
7.0
End!


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.permutation(np.arange(32*3)))
dataset = dataset.batch(32)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
sess = tf.InteractiveSession()
try:
    while True:
        print(sess.run(one_element))
except tf.errors.OutOfRangeError:
    print("End!")

[40  4 87 58 84 79 93 90 24  6 88 32 43 36 53 81 31 14 45 52 12 61 21 85
 38 22 20 69 29 65 17 44]
[92  9 73 62  2  5 10 82 68 25 37 13 49 57 89 18 95 46 48 33 72 86 63 26
 91  8  7 56 51 41 11 94]
[55 16 47 35 15 50 39 19 23 74  1 34 60 76 77 66 75  0 30 59 28 78  3 67
 70 80 42 71 54 83 64 27]
End!


In [43]:
limit = tf.placeholder(dtype=tf.int32, shape=[])
dataset = tf.data.Dataset.from_tensor_slices(tf.range(start=0, limit=limit))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={limit: 10})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

In [48]:
dataset = tf.data.Dataset.from_tensor_slices(
    {"a": tf.random_uniform([4]), 
     "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)
print(dataset.output_shapes)

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


In [14]:
iterator = tf.data.Iterator.from_structure(tf.int64, tf.TensorShape([]))
dataset_range = tf.data.Dataset.range(10)
range_initializer = iterator.make_initializer(dataset_range)
dataset_evens = dataset_range.filter(lambda x: x % 2 == 0)
evens_initializer = iterator.make_initializer(dataset_evens)

def model_fn(a):
    return a * 2, a * 10

sess = tf.InteractiveSession()
prediction, loss = model_fn(iterator.get_next())
for _ in range(3):
    sess.run(range_initializer)  ####
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break
    
    sess.run(evens_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print("twooooo")
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break

0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90
0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90
0 0
2 10
4 20
6 30
8 40
10 50
12 60
14 70
16 80
18 90


In [15]:
#　iterator = tf.data.Iterator.from_structure(tf.int32, tf.TensorShape([None]))
# dataset_range = tf.data.Dataset.range(10)

dataset_range = tf.data.Dataset.from_tensor_slices(np.arange(10, dtype=np.int32))
dataset_range = dataset_range.batch(2)
iterator = tf.data.Iterator.from_structure(dataset_range.output_types, dataset_range.output_shapes)
range_initializer = iterator.make_initializer(dataset_range)
dataset_evens = dataset_range.filter(lambda x: x % 2 == 0)
evens_initializer = iterator.make_initializer(dataset_evens)

def model_fn(a):
    return a * 2, a * 10

sess = tf.InteractiveSession()
prediction, loss = model_fn(iterator.get_next())
for _ in range(1):
    sess.run(range_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break
    
    sess.run(evens_initializer)
    while True:
        try:
            pred, loss_val = sess.run([prediction, loss])
            print("twooooo")
            print(pred, loss_val)
        except tf.errors.OutOfRangeError:
            break

[0 2] [ 0 10]
[4 6] [20 30]
[ 8 10] [40 50]
[12 14] [60 70]
[16 18] [80 90]


In [5]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

sess = tf.Session()
# sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value
    print(value)

0
1
2
3
4
5
6
7
8
9


## FeatureBuilder

In [163]:
class FeatureBuilder:
    def __init__(self, value_sharing=False, include_user_item=True,
                 n_users=None, n_items=None):
        self.value_sharing = value_sharing
        self.include_user_item = include_user_item
        self.n_users = n_users
        self.n_items = n_items

    def fit(self, categorical_features, numerical_features, train_size,
            user_features=None, item_features=None):
        self.total_count = 0  # add user & item indices before/after
        feature_indices = []
        feature_values = []
        for k, v in numerical_features.items():
            feature_indices.append([self.total_count] * train_size)
            feature_values.append(v)
            self.total_count += 1

        self.val_index_dict = defaultdict(dict)
        for k, v in categorical_features.items():
            unique_vals, indices = np.unique(v, return_inverse=True)
            unique_vals_length = len(unique_vals)
            indices += self.total_count
            self.val_index_dict[k].update(zip(unique_vals, np.unique(indices)))
            feature_indices.append(indices.tolist())
            feature_values.append([1.0] * train_size)
            self.total_count += unique_vals_length

        self.feature_size = self.total_count  # preserve total_count for transform function
        if self.include_user_item:
            feature_indices.append(user_features + self.feature_size)
            self.feature_size += self.n_users
            feature_indices.append(item_features + self.feature_size)
            self.feature_size += self.n_items
            feature_values.append([1.0] * train_size)
            feature_values.append([1.0] * train_size)

        feature_indices = np.array(feature_indices).T.astype(np.int32)
        feature_values = np.array(feature_values).T.astype(np.float32)
        return feature_indices, feature_values, self.feature_size

    def transform(self, test_cat_feat, test_num_feat, test_size,
                  test_user_features=None, test_item_features=None):
        test_feature_indices = []
        test_feature_values = []
        test_total_count = 0
        for k, v in test_num_feat.items():
            test_feature_indices.append([test_total_count] * test_size)
            test_feature_values.append(v)
            test_total_count += 1

        for k, v in test_cat_feat.items():
            indices = pd.Series(v).map(self.val_index_dict[k])
            indices = indices.fillna(self.feature_size)
            test_feature_indices.append(indices.tolist())
            test_feature_values.append([1.0] * test_size)

        if self.include_user_item:
            test_feature_indices.append(test_user_features + self.total_count)
            test_feature_indices.append(test_item_features + self.total_count + self.n_users)
            test_feature_values.append([1.0] * test_size)
            test_feature_values.append([1.0] * test_size)

        test_feature_indices = np.array(test_feature_indices).T.astype(np.int32)
        test_feature_values = np.array(test_feature_values).T.astype(np.float32)
        return test_feature_indices, test_feature_values

In [4]:
cc = {3: ['a', 'b', 'c'], 4: ['b', 'b', 'd']}
nn = {0: [1, 2, 3], 1: [4, 5, 6]}

In [28]:
cc2 = {3: ['a', 'b', 'a', 'r', 'q'], 4: ['d', 'b', 'd', 's', 'z']}
nn2 = {0: [1, 3, 3, 4, 5], 1: [4, 5, 6, 6, 4]}

In [17]:
uff = np.array([100, 200, 300])
iff = np.array([50, 100, 200])
n_users = 400
n_items = 600

In [32]:
uff2 = np.array([0,1,2,3,4])
iff2 = np.array([0,1,2,3,700])

In [19]:
fb = FeatureBuilder(n_users=n_users, n_items=n_items)

In [20]:
fi, fv, fs = fb.fit(cc, nn, 3, uff, iff)

In [21]:
fi

array([[  0,   1,   2,   5, 107, 457],
       [  0,   1,   3,   5, 207, 507],
       [  0,   1,   4,   6, 307, 607]], dtype=int32)

In [22]:
fv

array([[1., 4., 1., 1., 1., 1.],
       [2., 5., 1., 1., 1., 1.],
       [3., 6., 1., 1., 1., 1.]], dtype=float32)

In [23]:
fs

1007

In [33]:
fi2, fv2 = fb.transform(cc2, nn2, 5, uff2, iff2)

In [34]:
fi2

array([[   0,    1,    2,    6,    7,  407],
       [   0,    1,    3,    5,    8,  408],
       [   0,    1,    2,    6,    9,  409],
       [   0,    1, 1007, 1007,   10,  410],
       [   0,    1, 1007, 1007,   11, 1107]], dtype=int32)

In [35]:
fv2

array([[1., 4., 1., 1., 1., 1.],
       [3., 5., 1., 1., 1., 1.],
       [3., 6., 1., 1., 1., 1.],
       [4., 6., 1., 1., 1., 1.],
       [5., 4., 1., 1., 1., 1.]], dtype=float32)

In [164]:
class Dataset:
    def __init__(self, include_features=False):
        self.train_user = defaultdict(dict)
        self.train_item = defaultdict(dict)
        self.user2id = dict()
        self.item2id = dict()
        self.id2user = dict()
        self.id2item = dict()
        self.train_user_indices = list()
        self.train_item_indices = list()
        self.train_labels = list()
        self.test_user_indices = list()
        self.test_item_indices = list()
        self.test_labels = list()
        self.include_features = include_features
        if self.include_features:
            self.train_categorical_features = defaultdict(list)
            self.train_numerical_features = defaultdict(list)
            self.test_categorical_features = defaultdict(list)
            self.test_numerical_features = defaultdict(list)

    def _get_pool(self, data_path="../ml-1m/ratings.dat", shuffle=True, length="all",
                    train_frac=0.8, sep=",", user_col=None, item_col=None, seed=42):
        np.random.seed(seed)
        user_pool = set()
        item_pool = set()
        loaded_data = open(data_path, 'r').readlines()
        if shuffle:
            loaded_data = np.random.permutation(loaded_data)
        if length == "all":
            length = len(loaded_data)
        for i, data in enumerate(loaded_data[:length]):
            line = data.split(sep)
            user = line[user_col]
            item = line[item_col]
            if i <= int(train_frac * length):
                user_pool.add(user)
                item_pool.add(item)
        return user_pool, item_pool, loaded_data

    def build_dataset(self, data_path="../ml-1m/ratings.dat", shuffle=True, length="all",
                      train_frac=0.8, implicit_label=False, build_negative=False, seed=42,
                      num_neg=None, sep=",", user_col=None, item_col=None, label_col=None,
                      numerical_col=None, categorical_col=None):  # numerical feature 不做 embedding
        user_pool, item_pool, loaded_data = self._get_pool(data_path=data_path,
                                                             shuffle=shuffle,
                                                             length=length,
                                                             train_frac=train_frac,
                                                             sep=sep,
                                                             user_col=user_col,
                                                             item_col=item_col,
                                                             seed=seed)

        index_user = 0
        index_item = 0
        if length == "all":
            length = len(loaded_data)
        for i, data in enumerate(loaded_data[:length]):
            line = data.split(sep)
            user = line[user_col]
            item = line[item_col]
            label = line[label_col]
            try:
                user_id = self.user2id[user]
            except KeyError:
                user_id = index_user
                self.user2id[user] = index_user
                index_user += 1
            try:
                item_id = self.item2id[item]
            except KeyError:
                item_id = index_item
                self.item2id[item] = index_item
                index_item += 1

            if user not in user_pool or item not in item_pool:
                continue

            elif i <= int(train_frac * length):
                self.train_user_indices.append(user_id)
                self.train_item_indices.append(item_id)
                self.train_labels.append(int(label))
                self.train_user[user_id].update(dict(zip([item_id], [int(label)])))
                self.train_item[item_id].update(dict(zip([user_id], [int(label)])))

                if categorical_col is not None and self.include_features:
                    for cat_feat in categorical_col:
                        self.train_categorical_features[cat_feat].append(line[cat_feat])

                if numerical_col is not None and self.include_features:
                    for num_feat in numerical_col:
                        self.train_numerical_features[num_feat].append(line[num_feat])

            else:
                self.test_user_indices.append(user_id)
                self.test_item_indices.append(item_id)
                self.test_labels.append(int(label))

                if categorical_col is not None and self.include_features:
                    for cat_feat in categorical_col:
                        self.test_categorical_features[cat_feat].append(line[cat_feat])

                if numerical_col is not None and self.include_features:
                    for num_feat in numerical_col:
                        self.test_numerical_features[num_feat].append(line[num_feat])

        self.train_user_indices = np.array(self.train_user_indices)
        self.train_item_indices = np.array(self.train_item_indices)
        self.train_labels = np.array(self.train_labels)
        if self.include_features:
            fb = FeatureBuilder(include_user_item=True, n_users=self.n_users, n_items=self.n_items)
            self.train_feat_indices, self.train_feat_values, self.feature_size = \
                fb.fit(self.train_categorical_features,
                       self.train_numerical_features,
                       len(self.train_labels),
                       self.train_user_indices,
                       self.train_item_indices)

        # user_embedding, item_embedding, feature_embedding
        # np.unique(return_inverse=True)
        # numerical min_max_scale
        # min_occurance

        if implicit_label:
            self.train_labels = np.ones(len(self.train_labels), dtype=np.float32)

        if build_negative:
            self.build_trainset_implicit(num_neg)

        print("testset size before: ", len(self.test_labels))
    #    test_all = np.concatenate([np.expand_dims(self.test_user_indices, 1),
    #                               np.expand_dims(self.test_item_indices, 1),
    #                               np.expand_dims(self.test_labels, 1)],
    #                               axis=1)
    #    test_safe = test_all[(test_all[:, 0] < self.n_users) & (test_all[:, 1] < self.n_items)]
    #    test_danger = test_all[(test_all[:, 0] >= self.n_users) & (test_all[:, 1] >= self.n_items)]
    #    self.test_user_indices = test_safe[:, 0]
    #    self.test_item_indices = test_safe[:, 1]
    #    self.test_labels = test_safe[:, 2]

        self.test_user_indices = np.array(self.test_user_indices)
        self.test_item_indices = np.array(self.test_item_indices)
        self.test_labels = np.array(self.test_labels)
        if self.include_features:
            self.test_feat_indices, self.test_feat_values = \
                fb.transform(self.test_categorical_features,
                       self.test_numerical_features,
                       len(self.test_labels),
                       self.test_user_indices,
                       self.test_item_indices)

        if implicit_label:
            self.test_labels = np.ones(len(self.test_labels), dtype=np.float32)

        if build_negative:
            self.build_testset_implicit(num_neg)
    #        self.neg = negative_sampling(self, 4, self.batch_size)
        #    self.build_trainset_implicit()
        #    self.build_testset_implicit()
        print("testset size after: ", len(self.test_labels))
        return self
    
    @property
    def n_users(self):
        return len(self.train_user)

    @property
    def n_items(self):
        return len(self.train_item)

In [192]:
ratings = pd.read_csv("./ml-1m/ratings.dat", sep="::", header=None, names=['user', 'item', 'rating','timestamp'])
users = pd.read_csv("./ml-1m/users.dat", sep="::", header=None, names=['user', 'gender', 'age', 'occupation', 'zipcode'])
movies = pd.read_csv("./ml-1m/movies.dat", sep="::", header=None, names=['item', 'title', 'genres'])

In [10]:
ratings.head()

,user,item,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [22]:
users.head()

,user,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [14]:
movies.head()

,item,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
movies.shape

(3883, 3)

In [21]:
movies.genres.unique()

array(["Animation|Children's|Comedy", "Adventure|Children's|Fantasy",
       'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Crime|Thriller', "Adventure|Children's", 'Action',
       'Action|Adventure|Thriller', 'Comedy|Drama|Romance',
       'Comedy|Horror', "Animation|Children's", 'Drama',
       'Action|Adventure|Romance', 'Drama|Thriller', 'Drama|Romance',
       'Thriller', 'Action|Comedy|Drama', 'Crime|Drama|Thriller',
       'Drama|Sci-Fi', 'Romance', 'Adventure|Sci-Fi', 'Adventure|Romance',
       "Children's|Comedy|Drama", 'Documentary', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure', 'Crime|Thriller',
       "Animation|Children's|Musical|Romance", 'Action|Drama|Thriller',
       "Children's|Comedy", 'Drama|Mystery', 'Sci-Fi|Thriller',
       'Action|Comedy|Crime|Horror|Thriller', 'Drama|Musical',
       'Crime|Drama|Romance', 'Adventure|Drama', 'Action|Thriller',
       "Adventure|Children's|Comedy|Musical", 'Action|Drama|War',
       'Action|Adventur

In [193]:
def split_second(genre):
    try:
        return genre.split("|")[1]
    except IndexError:
        return "missing"
    
def split_third(genre):
    try:
        return genre.split("|")[2]
    except IndexError:
        return "missing"

In [194]:
movies['first_genre'] = movies.genres.apply(lambda x: x.split("|")[0])
movies['second_genre'] = movies.genres.apply(split_second)
movies['third_genre'] = movies.genres.apply(split_third)

In [195]:
movies.drop("genres", axis=1, inplace=True)
ratings.drop("timestamp", axis=1, inplace=True)
users.drop("zipcode", axis=1, inplace=True)

In [196]:
data = pd.merge(ratings, users, how="left", on="user")

In [197]:
data = pd.merge(data, movies, how="left", on="item")

In [198]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
data['title'] = oe.fit_transform(data.title.values.reshape(-1,1)).astype(object)

In [199]:
data.head(10)

,user,item,rating,gender,age,occupation,title,first_genre,second_genre,third_genre
0,1,1193,5,F,1,10,2452,Drama,missing,missing
1,1,661,3,F,1,10,1739,Animation,Children's,Musical
2,1,914,3,F,1,10,2289,Musical,Romance,missing
3,1,3408,4,F,1,10,1054,Drama,missing,missing
4,1,2355,5,F,1,10,557,Animation,Children's,Comedy
5,1,1197,3,F,1,10,2654,Action,Adventure,Comedy
6,1,1287,5,F,1,10,346,Action,Adventure,Drama
7,1,2804,5,F,1,10,671,Comedy,Drama,missing
8,1,594,4,F,1,10,3055,Animation,Children's,Musical
9,1,919,4,F,1,10,3656,Adventure,Children's,Drama


In [49]:
data.head(10)

,user,item,rating,gender,age,occupation,title,first_genre,second_genre,third_genre
0,1,1193,5,F,1,10,One Flew Over the Cuckoo's Nest (1975),Drama,missing,missing
1,1,661,3,F,1,10,James and the Giant Peach (1996),Animation,Children's,Musical
2,1,914,3,F,1,10,My Fair Lady (1964),Musical,Romance,missing
3,1,3408,4,F,1,10,Erin Brockovich (2000),Drama,missing,missing
4,1,2355,5,F,1,10,"Bug's Life, A (1998)",Animation,Children's,Comedy
5,1,1197,3,F,1,10,"Princess Bride, The (1987)",Action,Adventure,Comedy
6,1,1287,5,F,1,10,Ben-Hur (1959),Action,Adventure,Drama
7,1,2804,5,F,1,10,"Christmas Story, A (1983)",Comedy,Drama,missing
8,1,594,4,F,1,10,Snow White and the Seven Dwarfs (1937),Animation,Children's,Musical
9,1,919,4,F,1,10,"Wizard of Oz, The (1939)",Adventure,Children's,Drama


In [200]:
data.to_csv("ml-1m/merged_data.csv", header=False, index=False)

In [201]:
dataset = Dataset(include_features=True)

In [202]:
dataset.build_dataset("ml-1m/merged_data.csv", length="all", user_col=0, item_col=1, label_col=2, 
                     numerical_col=[4], categorical_col=[3,5,6,7,8,9])

testset size before:  200005
testset size after:  200005


In [203]:
dataset.train_feat_indices

array([[    0,     2,     7, ...,  3750,  3751,  9791],
       [    0,     1,    15, ...,  3750,  3752,  9792],
       [    0,     2,     3, ...,  3750,  3753,  9793],
       ...,
       [    0,     1,    18, ...,  3750,  5061, 12315],
       [    0,     2,     4, ...,  3750,  9690, 10318],
       [    0,     2,     3, ...,  3750,  6439,  9793]], dtype=int32)

In [204]:
dataset.test_feat_indices

array([[    0,     2,    19, ...,  3750,  5764, 11115],
       [    0,     1,     8, ...,  3750,  7849, 10401],
       [    0,     2,    21, ...,  3750,  6642, 10998],
       ...,
       [    0,     1,    11, ...,  3750,  3904, 10950],
       [    0,     2,    20, ...,  3745,  4186, 10515],
       [    0,     2,     3, ...,  3746,  7888, 10965]], dtype=int32)

In [205]:
dataset.train_feat_values

array([[25.,  1.,  1., ...,  1.,  1.,  1.],
       [45.,  1.,  1., ...,  1.,  1.,  1.],
       [25.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [18.,  1.,  1., ...,  1.,  1.,  1.],
       [35.,  1.,  1., ...,  1.,  1.,  1.],
       [18.,  1.,  1., ...,  1.,  1.,  1.]], dtype=float32)

In [206]:
dataset.train_feat_indices.shape, dataset.train_feat_indices.max()

((800168, 9), 13465)

In [207]:
dataset.feature_size

13466

In [208]:
cols = list(data.columns)
cols.remove('age')
cols.remove('rating')

In [209]:
cols

['user',
 'item',
 'gender',
 'occupation',
 'title',
 'first_genre',
 'second_genre',
 'third_genre']

In [218]:
total_cols = 0
for col in cols:
    print(data[col].nunique())

6040
3706
2
21
3706
18
18
16


In [229]:
len(np.unique(dataset.train_feat_indices[:,3]))

3675

In [212]:
data.dtypes

user             int64
item             int64
rating           int64
gender          object
age              int64
occupation       int64
title           object
first_genre     object
second_genre    object
third_genre     object
dtype: object

In [213]:
data.head(10)

,user,item,rating,gender,age,occupation,title,first_genre,second_genre,third_genre
0,1,1193,5,F,1,10,2452,Drama,missing,missing
1,1,661,3,F,1,10,1739,Animation,Children's,Musical
2,1,914,3,F,1,10,2289,Musical,Romance,missing
3,1,3408,4,F,1,10,1054,Drama,missing,missing
4,1,2355,5,F,1,10,557,Animation,Children's,Comedy
5,1,1197,3,F,1,10,2654,Action,Adventure,Comedy
6,1,1287,5,F,1,10,346,Action,Adventure,Drama
7,1,2804,5,F,1,10,671,Comedy,Drama,missing
8,1,594,4,F,1,10,3055,Animation,Children's,Musical
9,1,919,4,F,1,10,3656,Adventure,Children's,Drama
